In [1]:
import pandas as pd
import numpy as np

# make modules visible (in volt_finder)
import sys
sys.path.append('../')
import mygooglemaps as gg

In [2]:
# csv_file = 'raw_CircleElectrique_BornesAll.csv'
csv_file = 'cleaned_cs_locations_circuitElectrique.csv'
df = pd.read_csv(csv_file)
df.head(3)

,id,name,address,postal_code,charger_level,lat,lng,cost,gg_address,gg_lat,gg_lng,city,province
0,CEA-10804,FCR - Place Nelligan,"1134 St-René Boulevard West, Gatineau, QC",J8T 6H1,Level2,45.487851,-75.701220,"1,00 $ (CAD) de l'heure","1134 Boulevard Saint-René O, Gatineau, QC J8T ...",45.488182,-75.701358,Gatineau,QC
1,CEA-10097,DGI - Complexe Desjardins,"1251 rue Jeanne-Mance, Montréal, QC",H2X 3Y2,Level2,45.507924,-73.564140,"1,00 $ (CAD) de l'heure","1251 Rue Jeanne-Mance, Montréal, QC H2X, Canada",45.507039,-73.565129,Montréal,QC
2,CEA-11115,Municipalité de Saint-Adrien,"1589 Principale, Saint-Adrien, QC",J0A 1C0,Level2,45.8195769,-71.725003,"1,00 $ (CAD) de l'heure","1589 Rue Principale, Saint-Adrien, QC J0A 1C0,...",45.819577,-71.722814,Saint-Adrien,QC


### GeoCode address and parsing

In [335]:
# def geoCodethis(addr):
#     resp = gg.gmaps.geocode(addr)

#     try:
#         gg_address = resp[0]['formatted_address']
#         gg_lat = resp[0]['geometry']['location']['lat']
#         gg_lng = resp[0]['geometry']['location']['lng']
#         return gg_address, gg_lat, gg_lng
#     except:
#         return np.NaN, np.NaN, np.NaN

## REference to navigate response JSON
# street_num = resp[0]['address_components'][0]['long_name']
# long_name  = resp[0]['address_components'][1]['long_name']
# city       = resp[0]['address_components'][3]['long_name'] #not always present in that order [3]
# province   = resp[0]['address_components'][5]['long_name'] #not always present in that order [5]
# postal_code= resp[0]['address_components'][7]['long_name'] #not always present in that order [7]
# gg_address = resp[0]['formatted_address']
# lat        = resp[0]['geometry']['location']['lat']
# lng        = resp[0]['geometry']['location']['lng']

## Update all entries in DF 

In [3]:
# Drop null values
df = df[df.gg_address.notnull()]

#### 1. Apply formula Get google formated address, lat, and lng

In [4]:
df['gg_address'], df['gg_lat'], df['gg_lng'] = zip(*df['address'].apply(geoCodethis))

df.head()

#### 2. Parse address to get city, province, postal_code

In [345]:
def splitAddress(addr):  
    """ Takes a full address, returns city, province, postal_code"""
    city = np.NaN
    province = np.NaN
    postal_code = np.NaN
    
    split_addr = addr.split(', ')
    try:
        city = split_addr[1]
        province = split_addr[2][0:2]
        return city, province
    except:
        return np.NaN, np.NaN

In [346]:
df['city'], df['province'] = zip(*df['gg_address'].apply(splitAddress))
# df['city'], df['province'], df['postal_code'] = zip(*splitAddress(df['gg_address']))
df.head()

,Unnamed: 0,id,name,address,postal_code,charger_level,lat,lng,cost,gg_address,gg_lat,gg_lng,city,province
0,0,CEA-10804,FCR - Place Nelligan,"1134 St-René Boulevard West, Gatineau, QC",J8T 6H1,Level2,45.487851,-75.701220,"1,00 $ (CAD) de l'heure","1134 Boulevard Saint-René O, Gatineau, QC J8T ...",45.488182,-75.701358,Gatineau,QC
1,1,CEA-10097,DGI - Complexe Desjardins,"1251 rue Jeanne-Mance, Montréal, QC",H2X 3Y2,Level2,45.507924,-73.564140,"1,00 $ (CAD) de l'heure","1251 Rue Jeanne-Mance, Montréal, QC H2X, Canada",45.507039,-73.565129,Montréal,QC
2,2,CEA-11115,Municipalité de Saint-Adrien,"1589 Principale, Saint-Adrien, QC",J0A 1C0,Level2,45.8195769,-71.725003,"1,00 $ (CAD) de l'heure","1589 Rue Principale, Saint-Adrien, QC J0A 1C0,...",45.819577,-71.722814,Saint-Adrien,QC
3,3,CEC-00073,BRCC - CDSMML - Paspébiac,"5 Boul. Gérard-D-Lévesque Est, Pasbébiac, QC",G0C 2K0,FastDC,48.029571,-65.248140,"10,00 $ (CAD) de l'heure","5 Boulevard Gérard D. Levesque E, Paspébiac, Q...",48.029318,-65.248162,Paspébiac,QC
4,4,CEA-067,1482 | BHR | SLA | 2727 Thiemens : Bibliothèqu...,"2727 boul. Thimens, Saint-Laurent, QC",H4R 1T4,Level2,45.504816,-73.704340,"2,50 $ (CAD) par session","2727 Boulevard Thimens, Saint-Laurent, QC H4R ...",45.504146,-73.705375,Saint-Laurent,QC


### Create subset data

In [35]:
# Subset (Montreal)
mtl_all = df[df.city == 'Montréal']
print(len(mtl_all))

#Subset (Montreal and postal_code "H2")
mtl_h2 = mtl_all[mtl_all['postal_code'].str.startswith('H2')]
print(len(mtl_h2))

627
259


### Save cleaned data


In [44]:
mtl_all.columns

Index(['id', 'name', 'address', 'postal_code', 'charger_level', 'lat', 'lng',
       'cost', 'gg_address', 'gg_lat', 'gg_lng', 'city', 'province'],
      dtype='object')

In [51]:
def saveToCSVfunc(select_df, csvfile_name):
    """ Saves df in the right format and order of columns """
    sdf = select_df[['name', 'id', 'charger_level', 'cost', 'gg_address', 'city', \
                     'province', 'postal_code', 'gg_lat', 'gg_lng']]
    print(sdf.columns)
    sdf.to_csv(csvfile_name, index=False)

In [52]:
# Save a subset of of the data (MTL entries, with postal code starting in H2)
csvfile = 'cleaned_circuitElectrique_MTL_260.csv'
saveToCSVfunc(mtl_h2, csvfile)

Index(['name', 'id', 'charger_level', 'cost', 'gg_address', 'city', 'province',
       'postal_code', 'gg_lat', 'gg_lng'],
      dtype='object')


In [ ]:
# Save mtl_all
csvfile = 'cleaned_circuitElectrique_Montreal.csv'
saveToCSVfunc(mtl_all, csvfile)